Let $X_1$, $X_2$ be adjacent inputs into a segment on a DiPA. If $\Delta_i = X_1[i] - X_2[i]$, then let $\Delta_0 = 1$, $\Delta_\ell = -1$ for all $L$-transitions, and $\Delta_g = 1$ for all $G$-transitions.

What is the best coupling strategy for this pair of input sequences?

If we generalize coupling strategies on a segment as choices of $\gamma_0, \gamma_l, \gamma_g \in \mathbb{R}$ such that

$$
\begin{align}
x_0\langle 1 \rangle + \gamma_0 =^{#(\Delta_0 - \gamma_0)} &= x_0 \langle 2 \rangle \\
x_i\langle 1 \rangle + \gamma_\ell =^{#(\Delta_\ell - \gamma_\ell)} &= x_i \langle 2 \rangle \qquad \text{$i$th transition is $<$} \\
x_i\langle 1 \rangle + \gamma_g =^{#(\Delta_g - \gamma_g)} &= x_i \langle 2 \rangle \qquad \text{$i$th transition is $\geq$} \\
\end{align}
$$

such that $\gamma_\ell \leq \gamma_0 \leq \gamma_g$, the above forms a valid coupling strategy that ensures differential privacy with the bound

$$|\Delta_0 - \gamma_0| + \ell |\Delta_1 - \gamma_\ell| + m |\Delta_g - \gamma_g|$$

assuming $\ell$ transitions of type $<$ and $m$ transitions of type $\geq$.

Now, choosing $\gamma_0, \gamma_\ell$, and $\gamma_g$ is an optimization problem that can be formulated as a linear program:

$$\min_{\gamma_0, \gamma_\ell, \gamma_g \in \mathbb{R}} |\Delta_0 - \gamma_0| + \ell |\Delta_1 - \gamma_\ell| + m |\Delta_g - \gamma_g| $$ subject to the constraints

$$
\begin{align}
\gamma_\ell &\leq \gamma_0 \\
\gamma_g &\geq \gamma_0 \\
-1 \leq \gamma_\ell, \gamma_g, \gamma_0 &\leq 1\\
\end{align}
$$

In [2]:
import itertools

import cvxpy as cp

# Define variables
m = 1
l = 1
gamma_0 = cp.Variable()
gamma_l = cp.Variable()
gamma_g = cp.Variable()

# Solve the linear program above
objective = cp.Minimize(cp.abs(1 - gamma_0) + l * cp.abs(1 + gamma_l) + m * cp.abs(1 - gamma_g))
constraints = [gamma_l <= gamma_0, gamma_g >= gamma_0, -1 <= gamma_l, gamma_l <= 1, -1 <= gamma_g, gamma_g <= 1]

prob = cp.Problem(objective, constraints)

result = prob.solve()

print("gamma_0: ", gamma_0.value)
print("gamma_l: ", gamma_l.value)
print("gamma_g: ", gamma_g.value)

gamma_0:  0.9999999999166571
gamma_l:  -0.9999999999940027
gamma_g:  0.9999999999902367


Now, let's try something different, with the $\Delta$s being variable.

In [3]:
import cvxpy as cp
import itertools

def best_constraints_for(m, l, deltas):
    # Define variables
    gamma_0 = cp.Variable()
    gamma_l = cp.Variable()
    gamma_g = cp.Variable()

    # Solve the linear program above
    objective = cp.Minimize(cp.abs(deltas[0] - gamma_0) + l * cp.abs(deltas[1] - gamma_l) + m * cp.abs(deltas[2] - gamma_g))
    constraints = [gamma_l <= gamma_0, gamma_g >= gamma_0, -1 <= gamma_l, gamma_l <= 1, -1 <= gamma_g, gamma_g <= 1]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()
    return (result, gamma_0, gamma_l, gamma_g)

l = 10
m = 1
for deltas in itertools.product([-1, 1], repeat=3):
    result, gamma_0, gamma_l, gamma_g = best_constraints_for(m, l, deltas)
    print(deltas)
    print("gamma_0: ", gamma_0.value)
    print("gamma_l: ", gamma_l.value)
    print("gamma_g: ", gamma_g.value)
    print(result)

(-1, -1, -1)
gamma_0:  -0.9999999999890747
gamma_l:  -0.9999999999968856
gamma_g:  -0.9999999999346522
1.0741707523465038e-10
(-1, -1, 1)
gamma_0:  -0.9999999999443737
gamma_l:  -0.9999999999911517
gamma_g:  0.9999999999949891
1.4911971657483036e-10
(-1, 1, -1)
gamma_0:  0.9999999999920584
gamma_l:  0.9999999999907067
gamma_g:  0.9999999999935074
4.000000000078499
(-1, 1, 1)
gamma_0:  0.9999999999543566
gamma_l:  0.9999999999573385
gamma_g:  1.0000000000350975
2.000000000416069
(1, -1, -1)
gamma_0:  0.021102809768885986
gamma_l:  -0.9999999999924389
gamma_g:  0.021102809788911728
2.0000000000956364
(1, -1, 1)
gamma_0:  0.9999999999389524
gamma_l:  -0.999999999996396
gamma_g:  0.999999999983459
1.1362866203512567e-10
(1, 1, -1)
gamma_0:  1.00000000000746
gamma_l:  1.0000000000376856
gamma_g:  1.0000000001304028
2.000000000514719
(1, 1, 1)
gamma_0:  0.9999999999954462
gamma_l:  1.000000000000308
gamma_g:  0.9999999999937199
1.391364801150985e-11


Now, a new idea. Let's try formulating the search for a counterexample showing tightness as a linear program. The problem is defined as:

$$\max_{\Delta_i} \left(\min_{\gamma_i} m_0 |\Delta_0 - \gamma_0| + m_\ell |\Delta_\ell - \gamma_\ell| + m_g |\Delta_g - \gamma_g|\right)$$

subject to the constraints

$$
\begin{align}
\gamma_\ell &\leq \gamma_0 \\
\gamma_g &\geq \gamma_0 \\
-1 \leq \gamma_\ell, \gamma_g, \gamma_0 &\leq 1\\
-1 \leq \Delta_i &\leq 1 \qquad \forall i \in \{0, \ell, g\}
\end{align}
$$

In [4]:
def find_tight_counterexample(m: list[int]):
    """
    Find a coupling strategy that is tight for the given m = [m_0, m_l, m_g], where
    m_0 is the weight for the assignment transition, m_l is the weight for the L-transitions,
    and m_g is the weight for the G-transitions.
    :param m: The weights for the transitions.
    :return: A tuple ((delta_0, delta_l, delta_g), (gamma_0, gamma_l, gamma_g)) that form the counterexample and the coupling strategy.
    """
    # Define variables
    delta_0 = cp.Variable()
    delta_l = cp.Variable()
    delta_g = cp.Variable()
    gamma_0 = cp.Variable()
    gamma_l = cp.Variable()
    gamma_g = cp.Variable()
    t_0 = cp.Variable() # delta_0 - gamma_0
    t_l = cp.Variable() # delta_l - gamma_l
    t_g = cp.Variable() # delta_g - gamma_g

    # Solve the linear program above
    z = cp.Variable()

    objective = cp.Minimize(-z)
    constraints = [z <= m[0] * t_0 + m[1] * t_l + m[2] * t_g,

                   ] + [gamma_l <= gamma_0, gamma_g >= gamma_0, -1 <= gamma_l, gamma_l <= 1, -1 <= gamma_g, gamma_g <= 1, -1 <= delta_0, delta_0 <= 1, -1 <= delta_l, delta_l <= 1, -1 <= delta_g, delta_g <= 1, delta_0 - gamma_0 <= t_0, gamma_0 - delta_0 <= t_0, delta_l - gamma_l <= t_l, gamma_l - delta_l <= t_l, delta_g - gamma_g <= t_g, gamma_g - delta_g <= t_g]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()

    print("result: ", result)

    return (delta_0.value, delta_l.value, delta_g.value), (gamma_0.value, gamma_l.value, gamma_g.value)

m = [1, 1, 1]
(delta_0, delta_l, delta_g), (gamma_0, gamma_l, gamma_g) = find_tight_counterexample(m)
print("delta_0: ", delta_0)
print("delta_l: ", delta_l)
print("delta_g: ", delta_g)
print("gamma_0: ", gamma_0)
print("gamma_l: ", gamma_l)
print("gamma_g: ", gamma_g)

result:  -inf
delta_0:  None
delta_l:  None
delta_g:  None
gamma_0:  None
gamma_l:  None
gamma_g:  None


I asked ChatGPT to convert the above into a linear program through cvxpy. There is certainly something wrong, as I don't think this is really maximizing over $\Delta_i$ the minimum of the $\gamma_i$s.

Edit: Yikes, it turns out I am looking at a min-max problem, not a linear program. It looks like a bi-level optimisation problem, in which the inner problem is a linear program. The above will not work. Might have to try some iterative methods.

Going back to the `best_constraints_for` approach, I'll just try finding the best coupling strategies given particular delta.

In [5]:
import numpy as np
import cvxpy as cp

def best_constraints_for(m, l, deltas):
    # Define variables
    gamma_0 = cp.Variable()
    gamma_l = cp.Variable()
    gamma_g = cp.Variable()

    # Solve the linear program above
    objective = cp.Minimize(cp.abs(deltas[0] - gamma_0) + l * cp.abs(deltas[1] - gamma_l) + m * cp.abs(deltas[2] - gamma_g))
    constraints = [gamma_l <= gamma_0, gamma_g >= gamma_0, -1 <= gamma_l, gamma_l <= 1, -1 <= gamma_g, gamma_g <= 1]

    prob = cp.Problem(objective, constraints)

    result = prob.solve()
    return result, gamma_0, gamma_l, gamma_g

m = 50
l = 1
deltas = [1, -1, 1]
result, gamma_0, gamma_l, gamma_g = best_constraints_for(m, l, deltas)
print(f"Solution to the LP for l = {l}, m = {m}, and [delta_0,delta_l,delta_g] = {deltas}:")
print("gamma_0: ", gamma_0.value)
print("gamma_l: ", gamma_l.value)
print("gamma_g: ", gamma_g.value)
print(f"Optimal coupling bound on this input: {result}")

Solution to the LP for l = 1, m = 50, and [delta_0,delta_l,delta_g] = [1, -1, 1]:
gamma_0:  0.9999999994884569
gamma_l:  -0.9999999991641415
gamma_g:  0.9999999999824951
Optimal coupling bound on this input: 2.222645956173608e-09


This is where we are on tightness.

- $S^L$ is tight when there is an $L$-cycle, and $S^G$ is tight when there is a $G$-cycle.
- When there are only $<$ or $\geq$ transitions in a segment:
    - $S^J$ is tight when it is the least-cost strategy.
    - $S^L$/$S^G$ appears to be tight when it is the least-cost strategy.
    - I am not sure if $S^N$ is tight when it is least-cost.
- $S^J$ is not tight in general when there are both $<$ and $\geq$ transitions.

When I say least-cost above, I mean among the four strategies $S^G, S^L, S^N, S^J$ we currently know.

During the meeting with @Sky Li, we realized that coupling strategies can in general be parametrized by some value $\gamma$ that represents the difference between the coupled values of $x$ after the assignment transition. Later, I realized that by adding $\gamma_\ell$ and $\gamma_g$, two more values that represent differences between coupled values of `insample` after $<$ and $\geq$ transitions respectively, we can represent all four strategies.

The couplings that $S^J$ makes feel specifically tuned to segments with only L-transitions. From this, I thought it would be informative to explore this relaxed space of coupling strategies to explicitly find the parametrized least-cost strategy (in the entire space) given a segment.

From this, we might discover that there are only finitely many coupling strategies that are potentially optimal across all contexts, and we can go back to the discrete choice problem of which one to use. It is also possible that we find that the optimal coupling strategy varies continuously along this parameter space given individual segments. I will investigate this further.

In [6]:
from IPython.display import display, Latex, Math

def sgn(x: float) -> str:
    return '+' if x >= 0 else '-'

def fmt(x: float) -> str:
    return f"{sgn(x)} {abs(x):.3f}"

def fmtabs(x: float) -> str:
    return f"{abs(x):.3f}"

def print_known_coupling_strategies(m: int, l: int):
    display(Math(rf"\text{{The $S^L$ bound is {fmtabs(2 + 2 * m)} }} \\"
                 rf"\text{{The $S^G$ bound is {fmtabs(2 + 2 * l)} }} \\"
                 rf"\text{{The $S^N$ bound is {fmtabs(1 + l + m)} }} \\"
                 rf"\text{{The $S^J$ bound is {fmtabs(2 * l + 2 * m)} }}"
                 ))

def get_optimal_coupling_cost(m: int, l: int, deltas, display_output=False):
    result, gamma_0, gamma_l, gamma_g = best_constraints_for(m, l, deltas)
    gamma_0_val = np.round(gamma_0.value, 3)
    gamma_l_val = np.round(gamma_l.value, 3)
    gamma_g_val = np.round(gamma_g.value, 3)

    cost_0 = abs(deltas[0] - gamma_0_val)
    cost_l = abs(deltas[1] - gamma_l_val)
    cost_g = abs(deltas[2] - gamma_g_val)

    if display_output:
        display(Math(rf"\text{{Solution to the LP for $l$ = {l}, m = {m}, and [$\Delta_0, \Delta_\ell,\Delta_g$] = {deltas}:}} \\[1em]"
                 rf"x_0 \langle 1 \rangle {fmt(gamma_0_val)} = x_0 \langle 2 \rangle \qquad \text{{with cost}} |\Delta_0 - \gamma_0| = {fmtabs(cost_0)} \\ "
                 rf"x_\ell \langle 1 \rangle {fmt(gamma_l_val)} = x_\ell \langle 2 \rangle \qquad \text{{with cost}} |\Delta_\ell - \gamma_\ell| = {fmtabs(cost_l)} \\ "
                rf"x_g \langle 1 \rangle {fmt(gamma_g_val)} = x_g \langle 2 \rangle \qquad \text{{with cost}} |\Delta_g - \gamma_g| = {fmtabs(cost_g)} \\[1em]"
                 rf"\text{{The optimal coupling bound on this input is }} {fmtabs(cost_0)} + {fmtabs(cost_l)} l + {fmtabs(cost_g)}m = {fmtabs(cost_0 + m*cost_g + l*cost_l)}"
                 ))

    return result

m = 2
l = 2
max_res = -np.inf
worst_deltas = None
for deltas in itertools.product([-1, 0, 1], repeat=3):
    res = get_optimal_coupling_cost(m, l, deltas, display_output=False)
    if max_res < res:
        max_res = res
        worst_deltas = deltas
print_known_coupling_strategies(m, l)
get_optimal_coupling_cost(m, l, worst_deltas, display_output=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

4.000000000016456

In [10]:
get_optimal_coupling_cost(m, l, [0, 1, -1], display_output=True)

<IPython.core.display.Math object>

3.999999998576906